In [23]:
from tensorflow.keras.models import load_model
import pickle

def load_depression_classification_model(model_path, tokenizer_path, label_encoder_path):
    # Load the Keras model
    model = load_model(model_path)
    
    # Load the tokenizer
    with open(tokenizer_path, 'rb') as tokenizer_file:
        tokenizer = pickle.load(tokenizer_file)
        
    # Load the label encoder
    with open(label_encoder_path, 'rb') as label_encoder_file:
        label_encoder = pickle.load(label_encoder_file)
        
    return model, tokenizer, label_encoder

model, tokenizer, label_encoder = load_depression_classification_model('depression_model.h5', 'tokenizer.pkl', 'label_encoder.pkl')


In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from afinn import Afinn
def predict_depression_level(text, model, tokenizer, label_encoder):
    # Tokenize the text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=128)

    # Make prediction
    prediction = model.predict(padded_sequence)

    # Get the predicted class
    predicted_class = np.argmax(prediction)

    # Convert the class index back to original label
    predicted_level = label_encoder.inverse_transform([predicted_class])[0]

    return predicted_level


text = input("Enter text: ")



afinn = Afinn()
# Analyze text data and calculate sentiment scores
score=afinn.score(text)
if(score<0):
    predicted_level = predict_depression_level(text, model, tokenizer, label_encoder)
    print("Predicted depression level:", predicted_level)
else:
    print("you are fine")


Enter text: I'm consumed by thoughts of worthlessness and hopelessness. It's like I'm trapped in my own mind
1/1 [==============================] - 0s 30ms/step
Predicted depression level: Low


In [1]:
from tensorflow.keras.models import load_model
import pickle

def load_depression_classification_model_nrc(model_path, tokenizer_path, label_encoder_path):
    # Load the Keras model
    model = load_model(model_path)
    
    # Load the tokenizer
    with open(tokenizer_path, 'rb') as tokenizer_file:
        tokenizer = pickle.load(tokenizer_file)
        
    # Load the label encoder
    with open(label_encoder_path, 'rb') as label_encoder_file:
        label_encoder = pickle.load(label_encoder_file)
        
    return model, tokenizer, label_encoder

model, tokenizer, label_encoder = load_depression_classification_model_nrc('depression_model_NRClexicon.h5', 'tokenizer_nrc.pkl', 'label_encoder_nrc.pkl')


In [33]:
from nrclex import NRCLex
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
def predict_depression_level(text, model, tokenizer, label_encoder):
    # Tokenize the text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=128)
    # Make prediction
    prediction = model.predict(padded_sequence)
    # Decode the predicted label
    predicted_label = label_encoder.inverse_transform(prediction.argmax(axis=1))[0]
    return predicted_label

# Define a function to extract depression scores from the text
def extract_depression_scores(text):
    nrc_obj = NRCLex(text)
    raw_scores = nrc_obj.raw_emotion_scores
    sadness_score = raw_scores.get('sadness', 0)
    fear_score = raw_scores.get('fear', 0)
    anger_score = raw_scores.get('anger', 0)
    negative_score = raw_scores.get('negative', 0)
    disgust_score = raw_scores.get('disgust', 0)
    depression_score = sadness_score + fear_score + anger_score + negative_score + disgust_score
    return depression_score

# Get input text from the user
text = input("Enter text: ")

# Extract depression scores from the input text
depression_score = extract_depression_scores(text)

# Check if the depression score is zero
if depression_score != 0:
    # Predict the depression level using the trained model
    predicted_level = predict_depression_level(text, model, tokenizer, label_encoder)
    print("Predicted depression level:", predicted_level)
else:
    print("You are fine")

Enter text: I'm drowning in a sea of despair, and I don't know how to swim. It's like I'm being pulled further down with each passing day
1/1 [==============================] - 0s 32ms/step
Predicted depression level: Medium
